# 판례 일련 번호 불러오기

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as REQ
from xml.etree.ElementTree import parse
from urllib import parse
import random
import json

u1 = ('http://www.law.go.kr/DRF/lawSearch.do?OC=ID&search=2&target=prec&type=XML&display=100&page=')
u2 = ('http://www.law.go.kr/DRF/lawService.do?OC=ID&type=XML&target=prec&ID=')
wordList = ['친족', '면접교섭권', '약혼', '부양', '양육비', '이혼', '혼인', '위자료', '재산분할청구권', '입양', '파양', '친양자', '후견인']
numList = []

#판례 제목에 wordList의 단어가 들어간 판례 목록을 불러와 판례일련번호를 titleDic에 저장
for i in range(len(wordList)):
  t = 1
  w = parse.quote(wordList[i])
  while True:
    url = u1 + str(t) + '&query=' + w
    try:
      xml = REQ.urlopen(url).read()
      soup = BeautifulSoup(xml, "lxml-xml")
      nums = soup.select('판례일련번호')
      kinds = soup.select('사건종류명')
      if len(nums) == 0:
        break
      for k in range(len(nums)):
          if kinds[k].text == '민사':
                numList.append(nums[k].text)
    except:
      print('오류')
    t += 1

new_file = open('/content/drive/MyDrive/파일위치/numList_6.txt', "w", encoding='utf-8')
for n in numList:
  new_file.write(n+'\n')
new_file.close()


In [ ]:
import random
import pandas as pd

#랜덤하게 8천 개 뽑기

f = open('/content/drive/MyDrive/파일위치/numList_6_random.txt', 'r')
s = f.read()
numList = s.split('\n')
random.shuffle(numList)

pan = pd.read_pickle("/content/drive/MyDrive/파일위치/pan_6_random_1000.pkl")
numList2 = list(pan['number'])

for n in numList:
  if n in numList2:
    continue
  else:
    numList2.append(n)
  if len(numList2) == 8000:
    break


# 판례 본문 저장

In [ ]:
  from bs4 import BeautifulSoup
  import urllib.request as REQ
  from xml.etree.ElementTree import parse
  from urllib import parse
  import random
  import json
  from tqdm import tqdm
  u1 = ('http://www.law.go.kr/DRF/lawSearch.do?OC=ID&search=2&target=prec&type=XML&display=100&page=')
  u2 = ('http://www.law.go.kr/DRF/lawService.do?OC=ID&type=XML&target=prec&ID=')
  pansi_list = [] #판시사항 리스트
  art_list = [] #참조조문 리스트
  num_list = [] #일련번호 리스트
  panyo_list = [] #판결요지 리스트
  pan_list = [] #판례 내용(본문) 리스트
  precedentDic = {} #데이터 프레임으로 만들기 위한 딕셔너리

  for i in tqdm(range(len(numList2))):
    try:
                n = parse.quote(str(numList2[i]))
                url2 = u2 + n
                xml = REQ.urlopen(url2).read()
                soup = BeautifulSoup(xml, "lxml-xml")
                res = soup.find('판시사항').text
                res2 = soup.find('판결요지').text
                jo = soup.find('참조조문').text
                if res != ' ' and res2 != ' ':
                  num_list.append(str(numList2[i]))
                  pansi_list.append(res)
                  panyo_list.append(res2)
                    pan_list.append(soup.find('판례내용').text)
                else:
                  continue
                if jo != ' ':
                  art_list.append(jo)
                else:
                  art_list.append('')
                n_list.append(n)
    except Exception as e:
                print(e)
    if i % 100 == 99:
      precedentDic['number'] = num_list
      precedentDic['pansi'] = pansi_list
      precedentDic['panyo'] = panyo_list
      precedentDic['pan'] = panyo_list
      precedentDic['jo'] = art_list
      p_obj = pd.DataFrame(precedentDic, columns = ['number', 'pansi', 'panyo', 'jo'])
      p_obj.to_pickle("/content/drive/MyDrive/파일위치/pan_6_random_8000.pkl")
precedentDic['number'] = num_list
precedentDic['pansi'] = pansi_list
precedentDic['panyo'] = panyo_list
precedentDic['jo'] = art_list
p_obj = pd.DataFrame(precedentDic, columns = ['number', 'pansi', 'panyo', 'jo'])
p_obj.to_pickle("/content/drive/MyDrive/파일위치/pan_6_random_8000.pkl")


## 참조 조문에 민법이 있는 것만 저장

In [ ]:
jo_list = [] #각 판례 별로 참조 조문 리스트 저장(2차원)
for i in range(len(p_obj)):
  p = p_obj['jo'][i].split(',') #참조 조문 분리-[‘민법 제1조’, ‘민사소송법 제1조’] 이런 형태
  for p in p:
    for w in p.split():
      if not w.isalnum() and w.find('법') == -1:
        continue
    jo_list.append(w) #법 이름 저장
for i in range(len(p_obj)):
  jo = jo_list[i]
  c = 0 #민법이 있으면 0, 없으면 1
  for j in jo:
    if j == '민법':
      c = 1
       break
  if c == 0: #민법이 없으면 삭제
    p_obj = p_obj.drop(i)
p_obj.to_pickle("/content/drive/MyDrive/law/pan_6_random_8000.pkl")


# 민법 조항 저장

In [ ]:
import pandas as pd
f = open('/content/drive/MyDrive/파일위치/민법.txt', 'r')
s = f.read()
s = s.split('\n')  #민법 전체 텍스트를 한 줄 씩 분리한 리스트
jo = 1 #조항 번호(제 _조)
n_list = [] #조항 제목 리스트
a_list=[]#조항 내용 리스트
al = [] #각 조항 내용의 문장 리스트
check = 0 #해당 조항에 검색 단어 있으면 1, 없으면 0
for i in range(len(s)):
            #편이나 절, 장 등은 제외함. (삭제된 것들도)--제4편의 제2장이 너무 길어서 따로 조건에 추가함
            if (len(s[i]) < 12 and s[i].find('(') == -1 and s[i].find('.') == -1) or s[i].find('제2장') != -1 or s[i].find('제5편') != -1:
                #삭제된 조항이면 jo 증가
                if s[i].find('문헌') != -1:
                  if len(al) != 0:
                    a_list.append(al)
                  jo += 1
                  al = []
                continue
            if s[i].find('제1장') != -1 or s[i].find('제1절') != -1:
              continue
            str2 = '제' + str(jo+1) + '조'
            if s[i].find(str2) == 0 and s[i].find('(') != -1: #제(jo+1)조
                jo +=1
            str1 = '제' + str(jo) + '조'
            if s[i].find(str1) == 0 and s[i].find('(') != -1: ##제(jo)조
                if len(al) != 0:
                  a_list.append(al)
                  al = []
                n_list.append(s[i]) #조항 제목 추가
            else:
                al.append(s[i]) #조항 내용 추가
a_list.append(al)
aDic = {}
aDic['number'] = n_list
aDic['contents'] = a_list
a_obj = pd.DataFrame(aDic, columns = ['number', 'contents'])
a_obj.to_pickle("/content/drive/MyDrive/파일위치/article.pkl")
